# VacationPy
----




------------------------------------
- This code took about 18 seconds to run
- i've ran it a couple times and there seems to be a bias around africa, the middle east, and india

In [96]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from full_country import full_name_dict

### this allows me to keep my keys in a folder outside my repo
import sys
try:
    sys.path.insert(1,'/Users/Trav/Desktop/Coding/GitHub_Repositories/API_PACKAGE')
    from api_keys import g_key
except:
    from api_keys import g_key





### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [97]:
## import csv to dataframe, add try block to deal with directory issues
csv_import_path = os.path.join('..','Weather','Export Folder','city_data.csv')
city_weather_df=pd.read_csv(csv_import_path)

city_weather_df.head()

# # ### using this to limit the request, remove once completed
# city_weather_df=city_weather_df.head()
# city_weather_df.head(20)

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
0,-11.43,-91.52,san cristobal,ec,"(-0.9166667, -89.5666667)",-0.916667,-89.566667,67.24,95,86,10.04,67.24
1,34.58,-158.90,kapaa,us,"(22.0783333, -159.3219444)",22.078333,-159.321944,74.88,65,75,4.70,78.80
2,66.21,-28.87,olafsvik,is,"(64.883333, -23.716667)",64.883333,-23.716667,37.35,91,76,6.87,37.35
3,-9.39,-62.28,ariquemes,br,"(-9.933333, -63.066667)",-9.933333,-63.066667,72.32,91,100,4.63,72.32
4,52.76,177.19,beringovskiy,ru,"(63.05, 179.316667)",63.050000,179.316667,21.70,89,62,13.11,21.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [98]:
#configure gmaps with key
gmaps.configure(api_key=g_key)

In [99]:

#defines coordinates to plot and weight is humididty
### shoul these be .astype(float)
locations=city_weather_df[['Latitude','Longitude']]
weighty=city_weather_df['Humidity']

fig=gmaps.figure(center=(0,0),zoom_level=2)
heat=gmaps.heatmap_layer(locations,weights=weighty,dissipating=True,max_intensity=200,point_radius=20)
fig.add_layer(heat)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [100]:
#narrows down the dataframe to a dataframe with ideal weather
ideal_weather = city_weather_df.loc[(city_weather_df['Max Temp']>70)&(city_weather_df['Max Temp']<80)&(city_weather_df['Wind Speed']<10)&(city_weather_df['Cloudiness']==0),:]

#drops bad data
ideal_weather.dropna()


ideal_weather.head()

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
46,-22.83,-109.38,rikitea,pf,"(-23.1202778, -134.9691667)",-23.120278,-134.969167,76.01,67,0,4.00,76.01
209,29.34,34.52,elat,il,"(29.558049, 34.948212)",29.558049,34.948212,75.20,64,0,9.17,75.20
222,19.26,53.31,salalah,om,"(17.0175, 54.0827778)",17.017500,54.082778,77.00,36,0,4.70,77.00
237,-29.55,151.18,inverell,au,"(-29.774031, 151.111664)",-29.774031,151.111664,73.00,24,0,7.00,73.00
274,32.32,41.55,hit,iq,"(33.6366667, 42.8225)",33.636667,42.822500,71.15,20,0,3.31,71.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [101]:
#simplifies and defines the hotel_df
hotel_df=ideal_weather.drop(['Approximate Lat','Approximate Lon','Exact Coordinates'],axis=1).reset_index(drop=True)

#creates blank series that will be filled in
hotel_df['Hotel Name']=""
hotel_df['Hotel Lat']=""
hotel_df['Hotel Lon']=""
# #renaming and making uppercase
# hotel_df=hotel_df.rename(columns={'City Name':'City','Country Code':'Country'})
hotel_df['Country Code']=hotel_df['Country Code'].str.upper()

hotel_df=hotel_df.rename(columns={'City Name':'City'})
#creating a dataframe from the countries full names
country_name_df=pd.DataFrame.from_dict(full_name_dict,orient='index').reset_index()
country_name_df=country_name_df.rename(columns={'index':'Country Code',0:'Country'})

#merging in the country names

hotel_df=pd.merge(country_name_df,hotel_df,on="Country Code",how='right')

hotel_df.head()

,Country Code,Country,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,AU,Australia,inverell,-29.774031,151.111664,73.00,24,0,7.00,73.00,,,
1,EG,Egypt,aswan,24.087500,32.898889,76.82,52,0,3.80,76.82,,,
2,IL,Israel,elat,29.558049,34.948212,75.20,64,0,9.17,75.20,,,
3,IQ,Iraq,hit,33.636667,42.822500,71.15,20,0,3.31,71.15,,,
4,MG,Madagascar,tsihombe,-25.300000,45.483333,74.32,50,0,8.03,74.32,,,


In [102]:

### finds the closest hotel and fills in the hotel name and coordinates to the dataframe
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_radius = 5000
target_type = "Hotel"

for i,r in hotel_df.iterrows():
    c_lat=hotel_df['Latitude'][i]
    c_lon = hotel_df['Longitude'][i]
    target_coordinates =f'{c_lat}, {c_lon}' 
    
    param = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    places_json=requests.get(base_url,params=param).json()
    ## the city tessalit is literally in the middle of the sahara desert and returns no results, so this is a work around.

    if places_json['status']=='ZERO_RESULTS':
        hotel_df['Hotel Name'][i]=np.nan
    else:
        hotel_df['Hotel Name'][i]=places_json["results"][0]['name']
        hotel_df['Hotel Lat'][i]=places_json["results"][0]['geometry']['location']['lat']
        hotel_df['Hotel Lon'][i]=places_json["results"][0]['geometry']['location']['lng']


## Dropping the cities that returned no results 
hotel_df=hotel_df.dropna()

hotel_df.head(30)


,Country Code,Country,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,AU,Australia,inverell,-29.774031,151.111664,73.00,24,0,7.00,73.00,Cousins Motor Inn - 3.5 Star Rated,-29.7794,151.116
1,EG,Egypt,aswan,24.087500,32.898889,76.82,52,0,3.80,76.82,Sofitel Legend Old Cataract Aswan,24.0942,32.8974
2,IL,Israel,elat,29.558049,34.948212,75.20,64,0,9.17,75.20,"Al Manara, a Luxury Collection Hotel, Saraya A...",29.5358,34.9933
4,MG,Madagascar,tsihombe,-25.300000,45.483333,74.32,50,0,8.03,74.32,Restaurant Le Paradis du sud,-25.3187,45.4837
6,OM,Oman,salalah,17.017500,54.082778,77.00,36,0,4.70,77.00,HYATT SALALAH HOTEL,17.0194,54.1108
7,PF,French Polynesia,rikitea,-23.120278,-134.969167,76.01,67,0,4.00,76.01,Pension Maro'i,-23.1206,-134.969
9,ZM,Zambia,serenje,-13.233333,30.233333,70.84,48,0,9.93,70.84,SERENJE SOCH INN,-13.2251,30.2287


In [103]:
# # #create a new folder and exports the dataframe

def dataframe_to_csv(df):
    current_directory=os.getcwd()
    new_folder_path =os.path.join(current_directory,r'Export Folder')
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    csv_export_path = os.path.join(new_folder_path,'hotel_data.csv')
    df.to_csv(csv_export_path,index=False)
dataframe_to_csv(hotel_df)

In [104]:
## import the csv file you just saved
def import_path():
    current_directory=os.getcwd()
    import_path.new_folder_path =os.path.join(current_directory,r'Export Folder')
    csv_export_path = os.path.join(import_path.new_folder_path,'hotel_data.csv')
    return csv_export_path

im_path=import_path()
csv_imp_df=pd.read_csv(im_path,delimiter=',')
csv_imp_df.head()

,Country Code,Country,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,AU,Australia,inverell,-29.774031,151.111664,73.00,24,0,7.00,73.00,Cousins Motor Inn - 3.5 Star Rated,-29.779394,151.116486
1,EG,Egypt,aswan,24.087500,32.898889,76.82,52,0,3.80,76.82,Sofitel Legend Old Cataract Aswan,24.094242,32.897351
2,IL,Israel,elat,29.558049,34.948212,75.20,64,0,9.17,75.20,"Al Manara, a Luxury Collection Hotel, Saraya A...",29.535782,34.993276
3,MG,Madagascar,tsihombe,-25.300000,45.483333,74.32,50,0,8.03,74.32,Restaurant Le Paradis du sud,-25.318744,45.483670
4,OM,Oman,salalah,17.017500,54.082778,77.00,36,0,4.70,77.00,HYATT SALALAH HOTEL,17.019380,54.110750


In [105]:


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in csv_imp_df.iterrows()]

# hotel_info


In [106]:
### plotting heat map and hotel locations
y_locations=city_weather_df[['Latitude','Longitude']]
x_locations=csv_imp_df[['Hotel Lat','Hotel Lon']]
x_weighty=city_weather_df['Humidity']



x_fig=gmaps.figure(center=(0,0),zoom_level=2)
x_heat=gmaps.heatmap_layer(y_locations,weights=x_weighty,dissipating=True,max_intensity=200,point_radius=20)
x_marks=gmaps.marker_layer(x_locations,info_box_content=hotel_info)
x_fig.add_layer(x_marks)
x_fig.add_layer(x_heat)

x_fig

Figure(layout=FigureLayout(height='420px'))